In [3]:
df = spark.read.parquet("gs://bbm-labs-ds-news/bbm-news-with-content.parquet")
print df.count()

438934


In [4]:
%%time
## Before write the dataframe into cassandra, you have to first create a table with the same schema 
## ssh to the cassandra instance
## First type ```cqlsh```
## then type ```use bbmnews;```
## then here is an example to create the table 
##
## CREATE TABLE news_input(
##   article_id int,
##   date text,
##   article_url text, 
##   publisher_name text, 
##   channel_name text, 
##   total_views int,
##   title text,
##   image_caption text,
##   content text, 
##   language text,
##   PRIMARY KEY (article_id, date)
##);

# save whole dataframe into cassandra
df.write.format("org.apache.spark.sql.cassandra")\
    .mode('append')\
    .options(table="news_input", keyspace="bbmnews")\
    .save()

CPU times: user 4 ms, sys: 8 ms, total: 12 ms
Wall time: 58.7 s


In [5]:
%%time
# read dataframe from cassandra
df_new = spark.read.format("org.apache.spark.sql.cassandra")\
    .options(table="news_input", keyspace="bbmnews")\
    .load()
df_new.show()

+----------+--------------------+--------------------+------------+--------------------+--------------------+--------+--------------+--------------------+-----------+
|article_id|                date|         article_url|channel_name|             content|       image_caption|language|publisher_name|               title|total_views|
+----------+--------------------+--------------------+------------+--------------------+--------------------+--------+--------------+--------------------+-----------+
|    179343|2017-07-01 00:00:...|https://news.bbme...|        News|
Suara.com - Ment...|Menteri Koordinat...|      id|     Suara.com|Bagian dari ISIS,...|          1|
|    179343|2017-08-03 00:00:...|https://news.bbme...|        News|
Suara.com - Ment...|Menteri Koordinat...|      id|     Suara.com|Bagian dari ISIS,...|          1|
|    179343|2017-08-09 00:00:...|https://news.bbme...|        News|
Suara.com - Ment...|Menteri Koordinat...|      id|     Suara.com|Bagian dari ISIS,...|         12